# Project Template - adapted

## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

Be sure to start the stream on Kafka!

## Create Spark Session

In [2]:
spark = (
    SparkSession.builder
        .appName("Project DPA")
        .master("local[*]")   
        .config("spark.jars.packages",
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0")
        .config("spark.sql.repl.eagerEval.enabled", True)
        .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b444b75c-2162-4143-91d4-8c784b9714c6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 188ms :: artifacts dl 5m

## Schema Definiton

In [3]:
schema = StructType([
    StructField("name", StringType()),
    StructField("price", FloatType()),
    StructField("n_shares", IntegerType()),
    StructField("timestamp", StringType())
])

# Kafka Stream Ingestion

In [4]:
# Read from Kafka topic "stock"
raw_df = (
    spark.readStream
         .format("kafka")
         .option("kafka.bootstrap.servers", "kafka1:9092")
         .option("subscribe", "stock")
         .option("startingOffsets", "latest")
         .load()
)

# Kafka sends bytes → convert to string
json_df = raw_df.selectExpr("CAST(value AS STRING)")

# Parse JSON using the schema
parsed_df = json_df.select(
    from_json(col("value"), schema).alias("data")
).select("data.*")

# Convert timestamp string into actual timestamp type
parsed_df = parsed_df.withColumn(
    "timestamp", to_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss'Z'")
)

# Make the dataset available to all following tasks
parsed_df.createOrReplaceTempView("stocks_stream")

# Start Streaming Output

This is optional but just to make sure that Spark is actually receiving data

In [ ]:
query = (
    parsed_df.writeStream
             .format("console")
             .outputMode("append")
             .option("truncate", False)
             .start()
)

## The assignment starts here

Remark: We consider a day being 5 minutes so we do not have to wait an actual day for the code results. Hence, an hour corresponds to 12.5 seconds.

### Task 3

In [5]:
task3 = (
    parsed_df
        .withWatermark("timestamp", "10 minutes")
        .groupBy(window("timestamp", "5 minutes"), "name")
        .agg(
            sum("n_shares").alias("tot_shares_exchanged")   
        )
        .orderBy(desc("tot_shares_exchanged"))
        .limit(10)
)

task3_query = (
    task3.writeStream
         .format("console")
         .outputMode("complete")
         .option("truncate", False)
         .start()
)

25/11/24 21:40:33 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a3cdc641-3ba6-441b-a767-fad89571f96c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+----+--------------------+
|window                                    |name|tot_shares_exchanged|
+------------------------------------------+----+--------------------+
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|ADM |6645080             |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AEP |2682253             |
+------------------------------------------+----+--------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----+--------------------+
|window                                    |name|tot_shares_exchanged|
+------------------------------------------+----+--------------------+
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AAPL|137610123           |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|MU  |86979220            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|BAC |75503648            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AMD |23720135            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|ORCL|17472685            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|RF  |15554214            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|CHK |15084706            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|MRK |14504029            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|QCOM|13598689            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|HAL |13

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----+--------------------+
|window                                    |name|tot_shares_exchanged|
+------------------------------------------+----+--------------------+
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AAPL|137610123           |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|MU  |106253185           |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|BAC |75503648            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AMD |66683631            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|GE  |31886756            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AMAT|29613400            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|JNPR|20688921            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|ORCL|17472685            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|FOXA|16954404            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|RF  |15

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----+--------------------+
|window                                    |name|tot_shares_exchanged|
+------------------------------------------+----+--------------------+
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AAPL|202107346           |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|BAC |171640270           |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|MU  |133545870           |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AMD |66683631            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|CSCO|46307515            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|GE  |31886756            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AMAT|29613400            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|RF  |26427073            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|QCOM|21198342            |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|JNPR|20

...

-------------------------------------------
Batch: 460
-------------------------------------------
+------------------------------------------+----+--------------------+
|window                                    |name|tot_shares_exchanged|
+------------------------------------------+----+--------------------+
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|BAC |4277320472          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|BAC |4007815176          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AAPL|3475999553          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|AAPL|2508013794          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|GE  |1861008891          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|MU  |1558183821          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AMD |1553999369          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|GE  |1427502364          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|F   |1393122825          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|CSCO|

-------------------------------------------
Batch: 461
-------------------------------------------
+------------------------------------------+----+--------------------+
|window                                    |name|tot_shares_exchanged|
+------------------------------------------+----+--------------------+
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|BAC |4277320472          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|BAC |4093014971          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AAPL|3475999553          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|AAPL|2508013794          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|GE  |1897926958          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|MU  |1558183821          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AMD |1553999369          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|GE  |1427502364          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|F   |1424386763          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|CSCO|

-------------------------------------------
Batch: 462
-------------------------------------------
+------------------------------------------+----+--------------------+
|window                                    |name|tot_shares_exchanged|
+------------------------------------------+----+--------------------+
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|BAC |4277320472          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|BAC |4093014971          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AAPL|3475999553          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|AAPL|2508013794          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|GE  |1897926958          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|MU  |1558183821          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AMD |1553999369          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|GE  |1427502364          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|F   |1424386763          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|CSCO|

-------------------------------------------
Batch: 463
-------------------------------------------
+------------------------------------------+----+--------------------+
|window                                    |name|tot_shares_exchanged|
+------------------------------------------+----+--------------------+
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|BAC |4277320472          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|BAC |4093014971          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AAPL|3475999553          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|AAPL|2508013794          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|GE  |1897926958          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|MU  |1558183821          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AMD |1553999369          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|GE  |1427502364          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|F   |1424386763          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|CSCO|

-------------------------------------------
Batch: 464
-------------------------------------------
+------------------------------------------+----+--------------------+
|window                                    |name|tot_shares_exchanged|
+------------------------------------------+----+--------------------+
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|BAC |4277320472          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|BAC |4093014971          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AAPL|3475999553          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|AAPL|2508013794          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|GE  |1897926958          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|MU  |1558183821          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|AMD |1553999369          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|GE  |1427502364          |
|[2025-11-24 21:45:00, 2025-11-24 21:50:00]|F   |1424386763          |
|[2025-11-24 21:40:00, 2025-11-24 21:45:00]|CSCO|